
# Capstone Project - The Battle of the Neighborhoods (Week 2)

Applied Data Science Capstone by IBM/Coursera,This notebook will be mainly used for the capstone project：

## Finding Best Place to date with friends in BeiJing,China
========================================================================

### Introduction--Problem definition

   In a  multicultural city and huge city like BeiJing, China. it might be daunting to find out which places are best to date your friends comfortably and relex after working time , considering the traffic issues of the venues，maybe near the subway station is better in a high traffic city， Not only that, one needs to look into different factors like shoping easy，restaurants, cafes good eating taste and so on.
    
   Here can take me to a subway station that I often use as location information and I go to foursquare to explore the types of venues in the street, intuitively sort the data from the places I used to go, extract features, and help me to visualize the characteristics of the data
   
   After cluster the place to help us to learn about them for different purpose and find out the best place for the coming dating.
   
### Audiences 
  The People who live in Beijing and have subway transportation hobby want to find a suitable venue to have a gathering with his friends based on difference purpose/interesting


### Data Collection and Preparation

To solve the problem, we need to find the following data:

List of Neighbourhoods/subways station and their properties.
Latitude and Longitude of the neighbourhoods/subways.
Venue data of the neighbourhoods


To find the list of neighbourhoods,I used the csv file "beijingstation2.csv" located on the IBM Cloud Object Storage https://s3.eu-geo.objectstorage.service.networklayer.com', They were more than 20 neighbourhoods in the csv file. Since the coordinates were of boundaries, the centre coordinates of each neighbourhood have to be calculated according to the average of all latitudes and longitudes. This would be needed in order to plot the neighbourhood clusters.

Further more, we also need to draw a map for the place I often vistied with the help of the tool (http://www.gpsspg.com/maps.htm)

To find the venue data, Foursquare API was used. It would show the most popular venues in each neighbourhoods while using the central coordinates of the respective neighbourhood.


## 1. Download and Explore Dataset

In [2]:
from sklearn.cluster import KMeans
import numpy as np
import sys
import types
import pandas as pd
from ibm_botocore.client import Config
import ibm_boto3
import requests
def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
client_44d4310629d4414889cf904a19adba4c = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='UYdfinbjczjz-WXwtEzkxnc3lnPlW9rkJRJdqDgAQWhH',
    ibm_auth_endpoint="https://iam.bluemix.net/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.eu-geo.objectstorage.service.networklayer.com')

body = client_44d4310629d4414889cf904a19adba4c.get_object(Bucket='course-donotdelete-pr-phhvmbfasslhdb',Key='beijingstation2.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df_data_1 = pd.read_csv(body)
print(df_data_1.shape)
df_data_1.head(20)


(20, 3)


,Station,Latitude,Longitude
0,XIERQI,40.052243,116.306144
1,SHANGDI,40.032958,116.320519
2,WUDAOKOU,39.992833,116.337780
3,ZHICHUNLU,39.976424,116.340141
4,DAZHONGSI,39.966923,116.345126
5,XIZHIMEN,39.941856,116.353234
6,CHEGONGZHUANG,39.931445,116.356180
7,DONGZHIMEN,39.941255,116.433859
8,HUIXINXIJIENANKOU,39.976998,116.417644
9,SANYUANQIAO,39.960879,116.457055


In [3]:
!conda install -c conda-forge folium=0.5.0 --yes
import folium

print('Folium installed and imported!')

latitude = 40.032958
longitude = 116.320519
# create map and display it
sanfran_map = folium.Map(location=[latitude, longitude], zoom_start=10)

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.8.2

Please update conda by running

    $ conda update -n base -c defaults conda



# All requested packages already installed.

Folium installed and imported!


In [4]:
# instantiate a feature group for the incidents in the dataframe
incidents = folium.map.FeatureGroup()

# loop through the 100 crimes and add each to the incidents feature group
for lat, lng, station in zip(df_data_1.Latitude, df_data_1.Longitude,df_data_1.Station):
    incidents.add_child(
        folium.features.CircleMarker(
            [lat, lng],
            radius=5, # define how big you want the circle markers to be
            color='yellow',
            fill=True,
            fill_color='blue',
            fill_opacity=0.6
        )
    )

# add incidents to map
sanfran_map.add_child(incidents)

## 2. Explore Neighborhoods

In [5]:
CLIENT_ID = '2JU3FIKJ4ZXIFXDDC3X1HVDYJFB53OB4AXUDDZHY5VSTK2J4' # your Foursquare ID
CLIENT_SECRET = 'WLZ3UIKA2MR3L1ERNJYEEAMRM1QQX2GPACHHOLTM5OSZZTNZ' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100
#print('Your credentails:')
#print('CLIENT_ID: ' + CLIENT_ID)
#print('CLIENT_SECRET:' + CLIENT_SECRET)

def getNearbyVenues(names, latitudes, longitudes, radius=3000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)


In [6]:
Venues = getNearbyVenues(names=df_data_1['Station'],
                                   latitudes=df_data_1['Latitude'],
                                   longitudes=df_data_1['Longitude']
                                  )

XIERQI
SHANGDI
WUDAOKOU
ZHICHUNLU
DAZHONGSI
XIZHIMEN
CHEGONGZHUANG
DONGZHIMEN
HUIXINXIJIENANKOU
SANYUANQIAO
TUANJIEHU
CHAOYANGMEN
HAIDIANHUANGZHUANG
AOTIZHONGXIN
WANGJINGNAN
DONGSISHITIAO
XIDAN
SHUANGJING
QINGNIANLU
WEIGONGCUN


In [7]:
print(Venues.shape)
Venues

(1769, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,XIERQI,40.052243,116.306144,李记潮汕砂锅粥,40.041352,116.335103,Cantonese Restaurant
1,XIERQI,40.052243,116.306144,Shantou Baheli Hai's Beef Restaurant (汕头八合里海记牛肉店),40.040046,116.333920,Cantonese Restaurant
2,XIERQI,40.052243,116.306144,Starbucks (星巴克),40.052671,116.296030,Coffee Shop
3,XIERQI,40.052243,116.306144,Starbucks (星巴克),40.046375,116.292217,Coffee Shop
4,XIERQI,40.052243,116.306144,Haidilao Hot Pot (海底捞火锅),40.027357,116.305631,Hotpot Restaurant
...,...,...,...,...,...,...,...
1764,WEIGONGCUN,39.957765,116.323136,Sculpting in Time (雕刻时光),39.955710,116.305167,Café
1765,WEIGONGCUN,39.957765,116.323136,麻辣诱惑 Spice Spirit,39.977089,116.309981,Chinese Restaurant
1766,WEIGONGCUN,39.957765,116.323136,Carrefour (家乐福),39.979649,116.307309,Supermarket
1767,WEIGONGCUN,39.957765,116.323136,雕刻时光 Sculpting In Time,39.978723,116.308185,Café


In [8]:
Venues.groupby('Neighborhood').count()
#print('There are {} uniques categories.'.format(len(Venues['Venue Category'].unique())))

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
AOTIZHONGXIN,79,79,79,79,79,79
CHAOYANGMEN,100,100,100,100,100,100
CHEGONGZHUANG,100,100,100,100,100,100
DAZHONGSI,100,100,100,100,100,100
DONGSISHITIAO,100,100,100,100,100,100
DONGZHIMEN,100,100,100,100,100,100
HAIDIANHUANGZHUANG,100,100,100,100,100,100
HUIXINXIJIENANKOU,93,93,93,93,93,93
QINGNIANLU,25,25,25,25,25,25


In [9]:
print('There are {} uniques categories.'.format(len(Venues['Venue Category'].unique())))

There are 141 uniques categories.


## 3. Analyze Each Neighborhood

In [10]:
# one hot encoding
hot= pd.get_dummies(Venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
hot['Neighborhood'] = Venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [hot.columns[-1]] + list(hot.columns[:-1])
hot = hot[fixed_columns]

print(hot.shape)
hot.head()

(1769, 142)


,Neighborhood,American Restaurant,Anhui Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,BBQ Joint,...,Theater,Theme Park,Tiki Bar,Toy / Game Store,Turkish Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Xinjiang Restaurant,Yunnan Restaurant,Zhejiang Restaurant
0,XIERQI,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,XIERQI,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,XIERQI,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,XIERQI,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,XIERQI,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [11]:
hot_grouped = hot.groupby('Neighborhood').mean().reset_index()
hot_grouped

,Neighborhood,American Restaurant,Anhui Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,BBQ Joint,...,Theater,Theme Park,Tiki Bar,Toy / Game Store,Turkish Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Xinjiang Restaurant,Yunnan Restaurant,Zhejiang Restaurant
0,AOTIZHONGXIN,0.012658,0.000000,0.00,0.000000,0.000000,0.00,0.00,0.012658,0.000000,...,0.000000,0.00,0.00,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.000000
1,CHAOYANGMEN,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.00,0.000000,0.000000,...,0.010000,0.00,0.00,0.00,0.01,0.00,0.030000,0.010000,0.010000,0.000000
2,CHEGONGZHUANG,0.000000,0.000000,0.00,0.010000,0.000000,0.00,0.00,0.000000,0.010000,...,0.020000,0.00,0.00,0.00,0.00,0.00,0.020000,0.010000,0.010000,0.010000
3,DAZHONGSI,0.000000,0.000000,0.00,0.010000,0.000000,0.00,0.00,0.000000,0.010000,...,0.000000,0.00,0.00,0.00,0.00,0.00,0.000000,0.020000,0.000000,0.000000
4,DONGSISHITIAO,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.00,0.000000,0.000000,...,0.010000,0.00,0.00,0.00,0.00,0.01,0.020000,0.010000,0.010000,0.000000
5,DONGZHIMEN,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.00,0.000000,0.010000,...,0.010000,0.00,0.01,0.00,0.00,0.02,0.020000,0.010000,0.010000,0.000000
6,HAIDIANHUANGZHUANG,0.000000,0.000000,0.00,0.000000,0.000000,0.01,0.00,0.000000,0.000000,...,0.000000,0.00,0.00,0.00,0.00,0.02,0.000000,0.030000,0.000000,0.000000
7,HUIXINXIJIENANKOU,0.010753,0.000000,0.00,0.000000,0.010753,0.00,0.00,0.043011,0.010753,...,0.000000,0.00,0.00,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.000000
8,QINGNIANLU,0.040000,0.000000,0.04,0.000000,0.000000,0.00,0.00,0.000000,0.000000,...,0.000000,0.00,0.00,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.000000
9,SANYUANQIAO,0.010000,0.000000,0.00,0.000000,0.010000,0.00,0.00,0.020000,0.010000,...,0.000000,0.00,0.00,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.000000


In [12]:
hot_grouped.shape

(20, 142)

In [13]:
num_top_venues = 5

for hood in hot_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = hot_grouped[hot_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----AOTIZHONGXIN----
                  venue  freq
0    Chinese Restaurant  0.13
1                 Hotel  0.13
2           Coffee Shop  0.11
3  Fast Food Restaurant  0.06
4           Pizza Place  0.04


----CHAOYANGMEN----
                venue  freq
0               Hotel  0.17
1  Chinese Restaurant  0.06
2       Shopping Mall  0.06
3                Café  0.05
4             Brewery  0.04


----CHEGONGZHUANG----
                  venue  freq
0    Chinese Restaurant  0.12
1           Coffee Shop  0.10
2  Fast Food Restaurant  0.06
3                 Hotel  0.06
4           Pizza Place  0.05


----DAZHONGSI----
                  venue  freq
0  Fast Food Restaurant  0.20
1           Coffee Shop  0.09
2           Pizza Place  0.09
3                  Café  0.07
4    Chinese Restaurant  0.06


----DONGSISHITIAO----
                venue  freq
0               Hotel  0.12
1  Chinese Restaurant  0.06
2             Brewery  0.05
3                 Bar  0.04
4       Shopping Mall  0.04


----DONGZHI

In [14]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = hot_grouped['Neighborhood']

for ind in np.arange(hot_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(hot_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,AOTIZHONGXIN,Chinese Restaurant,Hotel,Coffee Shop,Fast Food Restaurant,Pizza Place,Hotpot Restaurant,Café,Park,Szechuan Restaurant,New American Restaurant
1,CHAOYANGMEN,Hotel,Shopping Mall,Chinese Restaurant,Café,Dumpling Restaurant,Italian Restaurant,Bar,Brewery,Peking Duck Restaurant,French Restaurant
2,CHEGONGZHUANG,Chinese Restaurant,Coffee Shop,Fast Food Restaurant,Hotel,Pizza Place,Café,Historic Site,Hotpot Restaurant,Department Store,Szechuan Restaurant
3,DAZHONGSI,Fast Food Restaurant,Pizza Place,Coffee Shop,Café,Chinese Restaurant,Sandwich Place,Bar,Clothing Store,Hotel,Hotpot Restaurant
4,DONGSISHITIAO,Hotel,Chinese Restaurant,Brewery,Japanese Restaurant,Café,Shopping Mall,Bar,Pizza Place,Szechuan Restaurant,Dumpling Restaurant
5,DONGZHIMEN,Hotel,Chinese Restaurant,Japanese Restaurant,Café,Brewery,Coffee Shop,Shopping Mall,Pizza Place,Szechuan Restaurant,Dumpling Restaurant
6,HAIDIANHUANGZHUANG,Chinese Restaurant,Fast Food Restaurant,Café,Sandwich Place,Coffee Shop,Pizza Place,Bakery,Bar,Korean Restaurant,Xinjiang Restaurant
7,HUIXINXIJIENANKOU,Fast Food Restaurant,Chinese Restaurant,Coffee Shop,Hotel,Pizza Place,Hotpot Restaurant,Asian Restaurant,Multiplex,Shopping Mall,Park
8,QINGNIANLU,Supermarket,Coffee Shop,Hotel,Clothing Store,American Restaurant,Noodle House,Park,Farm,Electronics Store,Department Store
9,SANYUANQIAO,Hotel,Japanese Restaurant,Bakery,Italian Restaurant,Park,Chinese Restaurant,Café,Shopping Mall,Cocktail Bar,Brewery


## 4. Cluster Neighborhoods
Run k-means to cluster the neighborhood into 5 clusters

In [15]:
kclusters = 5

Cluster = hot_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Cluster)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 1, 0, 4, 1, 1, 2, 0, 3, 1], dtype=int32)

In [16]:
Merged = df_data_1

# add clustering labels
Merged['Cluster Labels'] = kmeans.labels_

# merge hot_grouped with df_data-1 to add latitude/longitude for each neighborhood
Merged = Merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Station')

Merged # check the last columns!

,Station,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,XIERQI,40.052243,116.306144,0,Coffee Shop,Fast Food Restaurant,Chinese Restaurant,Hotel,Cantonese Restaurant,Pizza Place,Shopping Mall,Sporting Goods Shop,Asian Restaurant,Platform
1,SHANGDI,40.032958,116.320519,1,Fast Food Restaurant,Hotel,Coffee Shop,Chinese Restaurant,Cantonese Restaurant,Asian Restaurant,Sandwich Place,Japanese Restaurant,Café,Bus Stop
2,WUDAOKOU,39.992833,116.337780,0,Fast Food Restaurant,Café,Sandwich Place,Chinese Restaurant,Hotel,Coffee Shop,Bar,Korean Restaurant,Pizza Place,Bakery
3,ZHICHUNLU,39.976424,116.340141,4,Fast Food Restaurant,Sandwich Place,Chinese Restaurant,Coffee Shop,Café,Clothing Store,Pizza Place,Korean Restaurant,Bar,Xinjiang Restaurant
4,DAZHONGSI,39.966923,116.345126,1,Fast Food Restaurant,Pizza Place,Coffee Shop,Café,Chinese Restaurant,Sandwich Place,Bar,Clothing Store,Hotel,Hotpot Restaurant
5,XIZHIMEN,39.941856,116.353234,1,Fast Food Restaurant,Chinese Restaurant,Coffee Shop,Hotel,Pizza Place,Café,Bar,Hotpot Restaurant,Noodle House,Xinjiang Restaurant
6,CHEGONGZHUANG,39.931445,116.356180,2,Chinese Restaurant,Coffee Shop,Fast Food Restaurant,Hotel,Pizza Place,Café,Historic Site,Hotpot Restaurant,Department Store,Szechuan Restaurant
7,DONGZHIMEN,39.941255,116.433859,0,Hotel,Chinese Restaurant,Japanese Restaurant,Café,Brewery,Coffee Shop,Shopping Mall,Pizza Place,Szechuan Restaurant,Dumpling Restaurant
8,HUIXINXIJIENANKOU,39.976998,116.417644,3,Fast Food Restaurant,Chinese Restaurant,Coffee Shop,Hotel,Pizza Place,Hotpot Restaurant,Asian Restaurant,Multiplex,Shopping Mall,Park
9,SANYUANQIAO,39.960879,116.457055,1,Hotel,Japanese Restaurant,Bakery,Italian Restaurant,Park,Chinese Restaurant,Café,Shopping Mall,Cocktail Bar,Brewery


In [17]:
# create map
import matplotlib.cm as cm
import matplotlib.colors as colors
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Merged['Latitude'], Merged['Longitude'], Merged['Station'], Merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## 5. Examine Clusters
Examine each cluster and determine the discriminating venue categories that distinguish each cluster. Based on the defining categories, you can then assign a name to each cluster

In [18]:
##===============================================
##Cluster1
##===============================================
test0=Merged.loc[Merged['Cluster Labels'] == 0]
print(test0.shape)
test0


(6, 14)


,Station,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,XIERQI,40.052243,116.306144,0,Coffee Shop,Fast Food Restaurant,Chinese Restaurant,Hotel,Cantonese Restaurant,Pizza Place,Shopping Mall,Sporting Goods Shop,Asian Restaurant,Platform
2,WUDAOKOU,39.992833,116.337780,0,Fast Food Restaurant,Café,Sandwich Place,Chinese Restaurant,Hotel,Coffee Shop,Bar,Korean Restaurant,Pizza Place,Bakery
7,DONGZHIMEN,39.941255,116.433859,0,Hotel,Chinese Restaurant,Japanese Restaurant,Café,Brewery,Coffee Shop,Shopping Mall,Pizza Place,Szechuan Restaurant,Dumpling Restaurant
10,TUANJIEHU,39.933450,116.461705,0,Hotel,Italian Restaurant,Dumpling Restaurant,Shopping Mall,Brewery,Japanese Restaurant,Café,Bar,Mexican Restaurant,Coffee Shop
17,SHUANGJING,39.893557,116.461962,0,Hotel,Coffee Shop,Shopping Mall,Chinese Restaurant,Café,Italian Restaurant,Dumpling Restaurant,Fast Food Restaurant,New American Restaurant,Cocktail Bar
18,QINGNIANLU,39.923018,116.517454,0,Supermarket,Coffee Shop,Hotel,Clothing Store,American Restaurant,Noodle House,Park,Farm,Electronics Store,Department Store


In [19]:
##===============================================
##Cluster2
##===============================================
test1=Merged.loc[Merged['Cluster Labels'] == 1]
print(test1.shape)
test1

(7, 14)


,Station,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,SHANGDI,40.032958,116.320519,1,Fast Food Restaurant,Hotel,Coffee Shop,Chinese Restaurant,Cantonese Restaurant,Asian Restaurant,Sandwich Place,Japanese Restaurant,Café,Bus Stop
4,DAZHONGSI,39.966923,116.345126,1,Fast Food Restaurant,Pizza Place,Coffee Shop,Café,Chinese Restaurant,Sandwich Place,Bar,Clothing Store,Hotel,Hotpot Restaurant
5,XIZHIMEN,39.941856,116.353234,1,Fast Food Restaurant,Chinese Restaurant,Coffee Shop,Hotel,Pizza Place,Café,Bar,Hotpot Restaurant,Noodle House,Xinjiang Restaurant
9,SANYUANQIAO,39.960879,116.457055,1,Hotel,Japanese Restaurant,Bakery,Italian Restaurant,Park,Chinese Restaurant,Café,Shopping Mall,Cocktail Bar,Brewery
11,CHAOYANGMEN,39.924540,116.433433,1,Hotel,Shopping Mall,Chinese Restaurant,Café,Dumpling Restaurant,Italian Restaurant,Bar,Brewery,Peking Duck Restaurant,French Restaurant
12,HAIDIANHUANGZHUANG,39.976909,116.317043,1,Chinese Restaurant,Fast Food Restaurant,Café,Sandwich Place,Coffee Shop,Pizza Place,Bakery,Bar,Korean Restaurant,Xinjiang Restaurant
16,XIDAN,39.907769,116.374751,1,Historic Site,Hotel,Coffee Shop,Shopping Mall,Park,Café,Chinese Restaurant,Department Store,French Restaurant,Electronics Store


In [20]:
##===============================================
##Cluster3
##===============================================
test2=Merged.loc[Merged['Cluster Labels'] == 2]
print(test2.shape)
test2

(4, 14)


,Station,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,CHEGONGZHUANG,39.931445,116.356180,2,Chinese Restaurant,Coffee Shop,Fast Food Restaurant,Hotel,Pizza Place,Café,Historic Site,Hotpot Restaurant,Department Store,Szechuan Restaurant
13,AOTIZHONGXIN,39.986407,116.394155,2,Chinese Restaurant,Hotel,Coffee Shop,Fast Food Restaurant,Pizza Place,Hotpot Restaurant,Café,Park,Szechuan Restaurant,New American Restaurant
15,DONGSISHITIAO,39.933852,116.434333,2,Hotel,Chinese Restaurant,Brewery,Japanese Restaurant,Café,Shopping Mall,Bar,Pizza Place,Szechuan Restaurant,Dumpling Restaurant
19,WEIGONGCUN,39.957765,116.323136,2,Fast Food Restaurant,Coffee Shop,Pizza Place,Café,Chinese Restaurant,Clothing Store,Hotel,Asian Restaurant,Hotpot Restaurant,Sandwich Place


In [21]:
##===============================================
##Cluster4
##===============================================
test3=Merged.loc[Merged['Cluster Labels'] == 3]
print(test3.shape)
test3

(1, 14)


,Station,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,HUIXINXIJIENANKOU,39.976998,116.417644,3,Fast Food Restaurant,Chinese Restaurant,Coffee Shop,Hotel,Pizza Place,Hotpot Restaurant,Asian Restaurant,Multiplex,Shopping Mall,Park


In [22]:
##===============================================
##Cluster5
##===============================================
test4=Merged.loc[Merged['Cluster Labels'] == 4]
print(test4.shape)
test4

(2, 14)


,Station,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,ZHICHUNLU,39.976424,116.340141,4,Fast Food Restaurant,Sandwich Place,Chinese Restaurant,Coffee Shop,Café,Clothing Store,Pizza Place,Korean Restaurant,Bar,Xinjiang Restaurant
14,WANGJINGNAN,39.984704,116.482311,4,Café,Korean Restaurant,Coffee Shop,Chinese Restaurant,Hotel,American Restaurant,Japanese Restaurant,Art Gallery,Fast Food Restaurant,Public Art


In [29]:
##===============================================
##Cluster1 Count Catalogies
##===============================================
test0=test0.drop(['Station','Latitude','Longitude','Cluster Labels'],axis=1)
df2=pd.concat(test0.iloc[:,i] for i in range(test0.shape[1]))
#适当修改索引
df2.head(20)
df2.index=np.arange(len(df2))
print(df2.shape)
df2.value_counts()

(60,)


Coffee Shop                6
Hotel                      6
Café                       4
Chinese Restaurant         4
Shopping Mall              4
Pizza Place                3
Dumpling Restaurant        3
Fast Food Restaurant       3
Bar                        2
Japanese Restaurant        2
Italian Restaurant         2
Brewery                    2
New American Restaurant    1
Farm                       1
Asian Restaurant           1
American Restaurant        1
Cocktail Bar               1
Szechuan Restaurant        1
Noodle House               1
Clothing Store             1
Cantonese Restaurant       1
Bakery                     1
Sandwich Place             1
Mexican Restaurant         1
Korean Restaurant          1
Sporting Goods Shop        1
Park                       1
Electronics Store          1
Department Store           1
Platform                   1
Supermarket                1
dtype: int64

In [24]:
##===============================================
##Cluster2 Count Catalogies
##===============================================
test1=test1.drop(['Station','Latitude','Longitude','Cluster Labels'],axis=1)
df2=pd.concat(test1.iloc[:,i] for i in range(test1.shape[1]))
#适当修改索引
df2.index=np.arange(len(df2))
print(df2.shape)
df2.value_counts()

(70,)


Chinese Restaurant        7
Café                      7
Hotel                     6
Coffee Shop               5
Bar                       4
Fast Food Restaurant      4
Shopping Mall             3
Sandwich Place            3
Pizza Place               3
Park                      2
Brewery                   2
French Restaurant         2
Italian Restaurant        2
Hotpot Restaurant         2
Bakery                    2
Japanese Restaurant       2
Xinjiang Restaurant       2
Historic Site             1
Bus Stop                  1
Peking Duck Restaurant    1
Cocktail Bar              1
Asian Restaurant          1
Cantonese Restaurant      1
Electronics Store         1
Korean Restaurant         1
Dumpling Restaurant       1
Department Store          1
Noodle House              1
Clothing Store            1
dtype: int64

In [25]:
##===============================================
##Cluster3 Count Catalogies
##===============================================
test2=test2.drop(['Station','Latitude','Longitude','Cluster Labels'],axis=1)
df2=pd.concat(test2.iloc[:,i] for i in range(test2.shape[1]))
#适当修改索引
df2.index=np.arange(len(df2))
print(df2.shape)
df2.value_counts()

(40,)


Chinese Restaurant         4
Pizza Place                4
Hotel                      4
Café                       4
Hotpot Restaurant          3
Coffee Shop                3
Fast Food Restaurant       3
Szechuan Restaurant        3
Japanese Restaurant        1
Park                       1
Bar                        1
Historic Site              1
Brewery                    1
Asian Restaurant           1
Clothing Store             1
New American Restaurant    1
Department Store           1
Sandwich Place             1
Shopping Mall              1
Dumpling Restaurant        1
dtype: int64

In [28]:
##===============================================
##Cluster3 Count Catalogies
##===============================================
test3=test3.drop(['Station','Latitude','Longitude','Cluster Labels'],axis=1)
df2=pd.concat(test3.iloc[:,i] for i in range(test3.shape[1]))
#适当修改索引
df2.index=np.arange(len(df2))
print(df2.shape)
df2.value_counts()

(10,)


Fast Food Restaurant    1
Asian Restaurant        1
Shopping Mall           1
Hotpot Restaurant       1
Chinese Restaurant      1
Coffee Shop             1
Multiplex               1
Hotel                   1
Pizza Place             1
Park                    1
dtype: int64

In [27]:
##===============================================
##Cluster4 Count Catalogies
##===============================================
test4=test4.drop(['Station','Latitude','Longitude','Cluster Labels'],axis=1)
df2=pd.concat(test4.iloc[:,i] for i in range(test4.shape[1]))
#适当修改索引
df2.index=np.arange(len(df2))
print(df2.shape)
df2.value_counts()

(20,)


Korean Restaurant       2
Chinese Restaurant      2
Coffee Shop             2
Fast Food Restaurant    2
Café                    2
Japanese Restaurant     1
Clothing Store          1
Sandwich Place          1
Xinjiang Restaurant     1
Pizza Place             1
American Restaurant     1
Art Gallery             1
Public Art              1
Hotel                   1
Bar                     1
dtype: int64